In [ ]:
#lets try and use torch instead
from PIL import Image
import pathlib
import scipy.io
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import SGD
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader

In [ ]:
NUM_CLASSES = 196


In [ ]:
transforms = transforms.Compose(
[
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])
train = datasets.StanfordCars("cars", split="train", transform=transforms, download=True) #run these if its first time on REPO!
test = datasets.StanfordCars("cars", split="test", transform=transforms, download=True)

# train = datasets.StanfordCars("cars", split="train", transform=transforms)  #otherwise run these with the path to folder
# test = datasets.StanfordCars("cars", split="test", transform=transforms)

  0%|          | 0/330960 [00:00<?, ?it/s]

Extracting cars/stanford_cars/car_devkit.tgz to cars/stanford_cars


  0%|          | 0/979269282 [00:00<?, ?it/s]

Extracting cars/stanford_cars/cars_train.tgz to cars/stanford_cars
Using downloaded and verified file: cars/stanford_cars/car_devkit.tgz
Extracting cars/stanford_cars/car_devkit.tgz to cars/stanford_cars


  0%|          | 0/977350468 [00:00<?, ?it/s]

Extracting cars/stanford_cars/cars_test.tgz to cars/stanford_cars


  0%|          | 0/185758 [00:00<?, ?it/s]

In [ ]:
train_loader = DataLoader(train, batch_size = 50, shuffle = True)
test_loader = DataLoader(test, batch_size = 50, shuffle = True)

In [ ]:
resnet = models.resnet18(weights= "DEFAULT")
for param in resnet.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
#lets set the output layer to be our 196 classes
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet.fc = nn.Sequential(nn.Linear(512, 1024), nn.ReLU(), nn.Dropout(p=0.2), nn.Linear(1024, NUM_CLASSES)) #lets add a multi layer network with 20% dropout for regularization
resnet.to(device)
optimizer = SGD(resnet.parameters(), lr=0.001, momentum=0.9)

#device = torch.device('cpu')
criterion = nn.CrossEntropyLoss()

In [ ]:
print(device)

cuda:0


In [17]:
BATCH_SIZE = 50
EPOCHS = 50
print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_loader)

for epoch in range(1, EPOCHS+1):
    running_loss = 0.0
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')
    for batch_idx, (data_, target_) in enumerate(train_loader):
        data_, target_ = data_.to(device), target_.to(device)
        optimizer.zero_grad()
        outputs = resnet(data_)
        loss = criterion(outputs, target_)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==target_).item()
        total += target_.size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, EPOCHS, batch_idx, total_step, loss.item()))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
    batch_loss = 0
    total_t=0
    correct_t=0
    with torch.no_grad():
        resnet.eval()
        for data_t, target_t in (test_loader):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = resnet(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
        val_acc.append(100 * correct_t/total_t)
        val_loss.append(batch_loss/len(test_loader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t/total_t):.4f}\n')

        
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(resnet.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')
    resnet.train()

Epoch 1

Epoch [1/50], Step [0/163], Loss: 4.7423
Epoch [1/50], Step [20/163], Loss: 4.6804
Epoch [1/50], Step [40/163], Loss: 4.6164
Epoch [1/50], Step [60/163], Loss: 4.6952
Epoch [1/50], Step [80/163], Loss: 4.6090
Epoch [1/50], Step [100/163], Loss: 4.6281
Epoch [1/50], Step [120/163], Loss: 4.6201
Epoch [1/50], Step [140/163], Loss: 4.6760
Epoch [1/50], Step [160/163], Loss: 4.6418

train-loss: 4.6522, train-acc: 10.7318
validation loss: 4.6297, validation acc: 11.7523

Improvement-Detected, save-model
Epoch 2

Epoch [2/50], Step [0/163], Loss: 4.5468
Epoch [2/50], Step [20/163], Loss: 4.4492
Epoch [2/50], Step [40/163], Loss: 4.5772
Epoch [2/50], Step [60/163], Loss: 4.6775
Epoch [2/50], Step [80/163], Loss: 4.3965
Epoch [2/50], Step [100/163], Loss: 4.6413
Epoch [2/50], Step [120/163], Loss: 4.5295
Epoch [2/50], Step [140/163], Loss: 4.3983
Epoch [2/50], Step [160/163], Loss: 4.5166

train-loss: 4.5654, train-acc: 14.4155
validation loss: 4.5498, validation acc: 13.3939

Improve

Here we see an improvment in our best validation accuracy up to around 40%, again considering this is a relatively small pretrained model (Resnet18) getting 40% accuracy means the model is likly picking up large scale differences between makes of different manufacturers, which is better than I expected from such a simple model + classification head. 